# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saipan,15.1355,145.7010,28.39,83,40,5.14,MP,1729306571
1,1,waitangi,-43.9535,-176.5597,12.68,91,4,2.24,NZ,1729306572
2,2,sinabang,2.4803,96.3801,28.92,70,41,2.01,ID,1729306573
3,3,severo-yeniseyskiy,60.3725,93.0408,-5.04,93,98,1.67,RU,1729306574
4,4,bethel,41.3712,-73.4140,10.12,72,0,0.00,US,1729306576


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',  
    size='Humidity',  
    geo=True,  
    tiles='OSM', 
    frame_width=800,  
    frame_height=600, 
    color='Country',  
    alpha=0.6,  
    hover_cols=['City', 'Humidity']  
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Narrow down cities that fit criteria and drop any results with null values
city_temp_1 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp_2 = city_temp_1.loc[city_temp_1['Max Temp']>21,:]
city_wind_sp = city_temp_2.loc[city_temp_2['Wind Speed']<4.5,:]
city_cloudiness = city_wind_sp.loc[city_wind_sp['Cloudiness']==0,:]
city_cloudiness

# Drop any rows with null values
city_ideal = city_cloudiness.dropna()

# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,chilmari,25.5559,89.6715,26.40,73,0,4.08,BD,1729306592
23,23,antonio enes,-16.2325,39.9086,21.67,82,0,1.30,MZ,1729306598
60,60,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.51,GF,1729306644
61,61,cabo san lucas,22.8909,-109.9124,24.86,70,0,3.13,MX,1729306398
184,184,lazaro cardenas,17.9583,-102.2000,26.41,88,0,1.87,MX,1729306799
303,303,arish,31.1316,33.7984,21.69,64,0,3.36,EG,1729306949
311,311,san jose de jachal,-30.2406,-68.7469,22.65,54,0,1.49,AR,1729306958
318,318,markaz-e hukumat-e darweshan,31.1323,64.1934,21.82,19,0,1.31,AF,1729306969
320,320,iranshahr,27.2025,60.6848,25.19,16,0,3.09,IR,1729306972
358,358,choix,26.7092,-108.3219,24.45,54,0,1.03,MX,1729307021


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,chilmari,BD,25.5559,89.6715,73,
23,antonio enes,MZ,-16.2325,39.9086,82,
60,remire-montjoly,GF,4.9167,-52.2667,94,
61,cabo san lucas,MX,22.8909,-109.9124,70,
184,lazaro cardenas,MX,17.9583,-102.2000,88,
303,arish,EG,31.1316,33.7984,64,
311,san jose de jachal,AR,-30.2406,-68.7469,54,
318,markaz-e hukumat-e darweshan,AF,31.1323,64.1934,19,
320,iranshahr,IR,27.2025,60.6848,16,
358,choix,MX,26.7092,-108.3219,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"circle:{lon},{lat},{radius}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
chilmari - nearest hotel: No hotel found
antonio enes - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
arish - nearest hotel: No hotel found
san jose de jachal - nearest hotel: No hotel found
markaz-e hukumat-e darweshan - nearest hotel: No hotel found
iranshahr - nearest hotel: No hotel found
choix - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
mhamid - nearest hotel: No hotel found
ad dabbah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
18,chilmari,BD,25.5559,89.6715,73,No hotel found
23,antonio enes,MZ,-16.2325,39.9086,82,No hotel found
60,remire-montjoly,GF,4.9167,-52.2667,94,No hotel found
61,cabo san lucas,MX,22.8909,-109.9124,70,No hotel found
184,lazaro cardenas,MX,17.9583,-102.2000,88,No hotel found
303,arish,EG,31.1316,33.7984,64,No hotel found
311,san jose de jachal,AR,-30.2406,-68.7469,54,No hotel found
318,markaz-e hukumat-e darweshan,AF,31.1323,64.1934,19,No hotel found
320,iranshahr,IR,27.2025,60.6848,16,No hotel found
358,choix,MX,26.7092,-108.3219,54,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

%%capture --no-display

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)